## import

In [1]:
import argparse
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
import cv2
import numpy as np
import os
from os import path
import pandas as pd
import csv

## Preprocessing for Video

#### trian_image

























































































#### Meteor Data

## 動的背景差分

手順
1. 記事を元にして差分アルゴリズムをくむ．
2. 背景を黒にする．
3. 画像保存(メテオ30f, なし5f)
3. 平均輝度(ノイズ)でノイズの有無をラベルなし2値分類(3：ラメテオ，　2：ラノイズメテオ, 1:ラノイズ　＆　001, 0：ノイズ)
4. 
https://qiita.com/hitomatagi/items/f9d4d6b816d729132231

In [ ]:
import cv2
import numpy as np

# 定数定義
ESC_KEY = 27     # Escキー
INTERVAL= 33     # インターバル
FRAME_RATE = 30  # fps

#image
train_csv = pd.read_csv("../input/train_list.csv", header=None)
i = 0
count_row = 0
for  row in train_csv.iloc[:,0]:
    #print(path.exists('../input/train/%s' % row))
    cap = cv2.VideoCapture('../input/train/%s' % row)
    x=0
    backkk = 0
    while(cap.isOpened()):
        flag, frame = cap.read()
        if backkk == 0:
            # 最初のみ背景フレーム
            back_frame = np.zeros_like(frame, np.float32)
            backkk += 1
            
        if flag == False:
            break
        # 入力画像を浮動小数点型に変換
        f_frame = frame.astype(np.float32)
        # 差分計算
        diff_frame = cv2.absdiff(f_frame, back_frame)
        # 背景の更新
        cv2.accumulateWeighted(f_frame, back_frame, 0.025)
        
        #has meteor
        if train_csv.iloc[count_row,1] == 1 and x % 2 == 0:
            cv2.imwrite("../input/diff_train/"+"diff_%s.png" % str(i).zfill(7), diff_frame.astype(np.uint8))
            i += 1# ファイル名更新
            with open('../input/diff2_train.csv', 'a') as f:
                writer = csv.writer(f, lineterminator='\n')
                writer.writerow(["%s" % str("diff_%s.png" % str(i).zfill(7)),int(train_csv.iloc[count_row,1])])
        
        #hasnt meteor
        elif x > 0 and x % 30 == 0:
            cv2.imwrite("../input/diff_train/"+"diff_%s.png" % str(i).zfill(7), diff_frame.astype(np.uint8))
            #print('Save', "img_%s.png" % str(i).zfill(5))
            with open('../input/diff2_train.csv', 'a') as f:
                writer = csv.writer(f, lineterminator='\n')
                writer.writerow(["%s" % str("diff_%s.png" % str(i).zfill(7)),int(train_csv.iloc[count_row,1])])
            i += 1#ファイル名更新
        x+=1
    cap.release()
    count_row += 1

### 輝度平均

In [3]:
#imagedata
dif_train_csv = pd.read_csv("../input/diff2_train.csv", header=None)

#RGB dataframe

dfRGB = pd.DataFrame([], columns=['img_name','B', 'G', 'R'])

# 画像の読み込み(RGB)
for  row in dif_train_csv.iloc[:,0]:
    img = cv2.imread('../input/diff_train/%s' % row)
    B = np.average(img[0], axis = 0) # 軸(axis)を指定する。2次元配列でaxis =0のときは行方向の平均を
    G = np.average(img[1], axis = 0)
    R = np.average(img[2], axis = 0)
    B = B.mean()
    G = G.mean()
    R = R.mean()
    series = pd.Series([str(row), B, G, R], index=dfRGB.columns)
    dfRGB = dfRGB.append(series, ignore_index=True)

In [4]:
dfRGB = dfRGB.set_index('img_name')

#### 画素平均による2値化

謎エラー<br>
ググったけど原因わからず<br>
処理自体はできている

In [5]:
for index, row in dfRGB.iterrows():
    #print(row.sum())
    if (row[["B", "G", "R"]].sum()) <= (25):
        dfRGB.loc[index,"noise"] = 1       
    else:
        dfRGB.loc[index, "noise"] = 0

In [6]:
x = 0
for  row in dif_train_csv.iloc[:,0]:
    dfRGB.loc[row, "meteor"] = dif_train_csv.iloc[x,1]
    x+=1

In [7]:
dfRGB.head(50)

,B,G,R,noise,meteor
img_name,,,,,
diff_0000000.png,5.667708,5.528125,5.666667,1.0,0.0
diff_0000001.png,2.765625,2.704167,2.703125,1.0,0.0
diff_0000002.png,1.287500,1.189583,1.191667,1.0,0.0
diff_0000003.png,1.950000,1.876042,1.347917,1.0,0.0
diff_0000004.png,1.243750,1.077083,0.911458,1.0,0.0
diff_0000005.png,5.485417,5.397917,5.373958,1.0,0.0
diff_0000006.png,2.801042,2.621875,2.486458,1.0,0.0
diff_0000007.png,1.748958,1.744792,1.581250,1.0,0.0
diff_0000008.png,1.773958,1.565625,1.123958,1.0,0.0


### k-means法でクラスタリング

In [8]:
import os
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import IncrementalPCA
from sklearn.cluster import KMeans
 
np.random.seed(5)

In [9]:
#--------------------------------------------------------------------- 
def img_to_matrix(img):
    img_array = np.asarray(img)
 
    return img_array
 
def flatten_img(img_array):#平坦化
    s = img_array.shape[0] * img_array.shape[1] * img_array.shape[2]
    img_width = img_array.reshape(1, s)
 
    return img_width[0]
#---------------------------------------------------------------------  

dataset = []
#--------------------------------------------------------------------- 
for index, row in dfRGB.iterrows():
    #img = img.resize((int(1232/6), int(1754/6)), Image.BICUBIC)
    if row['noise'] == 1 and row['meteor'] == 1:
        img = Image.open("../input/diff_train/%s" % index)
        img = img_to_matrix(img)
        img = flatten_img(img)
 
        dataset.append(img)
#--------------------------------------------------------------------- 
dataset = np.array(dataset)
print(dataset.shape)
print("Dataset make done.")

(15062, 172800)
Dataset make done.


#### PCA主成分分析

In [10]:
n = dataset.shape[0]
batch_size = 180
ipca = IncrementalPCA(n_components=100)
 
for i in range(n//batch_size):
    r_dataset = ipca.partial_fit(dataset[i*batch_size:(i+1)*batch_size])
 
r_dataset = ipca.transform(dataset)
print(r_dataset.shape)
print("PCA done.")

(15062, 100)
PCA done.


In [11]:
df_meteor = dfRGB[(dfRGB["noise"] == 1) & (dfRGB["meteor"] == 1)]
df_meteor.head()

,B,G,R,noise,meteor
img_name,,,,,
diff_0000154.png,8.554167,8.301042,8.100000,1.0,1.0
diff_0000155.png,8.108333,7.870833,7.680208,1.0,1.0
diff_0000156.png,7.683333,7.461458,7.275000,1.0,1.0
diff_0000157.png,7.277083,7.113542,7.050000,1.0,1.0
diff_0000158.png,7.162500,6.927083,7.215625,1.0,1.0


In [12]:
df_meteor["meteor_in_meteor"] = 0

/home/iwailab-ml/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
# K-means clustering
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=5).fit(r_dataset)
labels = kmeans.labels_
print("K-means clustering done.")

for i in range(n_clusters):
    label = np.where(labels==i)[0]
    for j in label:
        df_meteor.iloc[j,5] = i 
        #print(j)
 
print("Image placing done.")

K-means clustering done.


/home/iwailab-ml/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Image placing done.


In [15]:
df_meteor.head(100)

,B,G,R,noise,meteor,meteor_in_meteor
img_name,,,,,,
diff_0000154.png,8.554167,8.301042,8.100000,1.0,1.0,0
diff_0000155.png,8.108333,7.870833,7.680208,1.0,1.0,0
diff_0000156.png,7.683333,7.461458,7.275000,1.0,1.0,0
diff_0000157.png,7.277083,7.113542,7.050000,1.0,1.0,0
diff_0000158.png,7.162500,6.927083,7.215625,1.0,1.0,0
diff_0000159.png,7.355208,7.218750,7.057292,1.0,1.0,0
diff_0000160.png,6.753125,6.680208,6.776042,1.0,1.0,0
diff_0000161.png,6.233333,6.386458,6.509375,1.0,1.0,0
diff_0000162.png,5.357292,5.448958,5.576042,1.0,1.0,0


In [16]:
df_meteor.to_csv("../input/df_meteor.csv")

In [30]:
im = Image.open("../input/diff_train/diff_0000310.png")
im.show()

### submit

In [44]:
import keras
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, list_pictures, load_img
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [41]:
#image
from sklearn.model_selection import train_test_split
(X_train, X_test,
 y_train, y_test) = train_test_split(
    df_meteor.index.values,df_meteor["meteor_in_meteor"], test_size=0.1, random_state=0,
)

In [46]:

# フォルダの中にある画像を順次読み込む
# カテゴリーは0から始める

X = []
Y = []

# 対象Aの画像
for index, row in df_meteor.iterrows():
    img = img_to_array(load_img("../input/diff_train/"+str(index), target_size=(64,64)))
    X.append(img)

# arrayに変換
X = np.asarray(X)
Y = np.asarray(df_meteor["meteor_in_meteor"])

In [47]:
# 画素値を0から1の範囲に変換
X = X.astype('float32')
X = X / 255.0

# クラスの形式を変換
Y = np_utils.to_categorical(Y, 2)

# 学習用データとテストデータ
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=111)

## modeling

In [49]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.optimizers import Adam

from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.utils import plot_model
from keras.callbacks import TensorBoard

from keras.utils import np_utils

In [51]:
# CNNを構築
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))       # クラスは2個
model.add(Activation('softmax'))

# コンパイル
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=5, epochs=200,
                   validation_data = (X_test, y_test), verbose = 0)

## submit_data

#### test_image

In [31]:
i = 0
count_row = 0
for  file in os.listdir("../input/test"):
    #print(path.exists("../input/test/"+"01%s.mpg" % str(i).zfill(4)))
    i +=1
    cap = cv2.VideoCapture("../input/test/"+"01%s.mpg" % str(i).zfill(4))
    x=0
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag == False:
            break
        if x % 6 == 0:
            cv2.imwrite("../input/image_train/"+"img_%s.png" % str(i).zfill(7), frame)
            #print('Save', "img_%s.png" % str(i).zfill(5))
            with open('../input/im_train.csv', 'a') as f:
                writer = csv.writer(f, lineterminator='\n')
                writer.writerow(["%s" % str("img_%s.png" % str(i).zfill(7)),int(train_csv.iloc[count_row,1])])
            i += 1
        x+=1
    cap.release()
    count_row += 1

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
with open('../input/submit.csv', 'a') as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(["01%s.mpg" % str(i).zfill(4)])